# Foco da Analise, primeiras impressões


Realizar uma clusterização com as seguintes colunas, para compreender quais são os perfis de carros que mais vendidos por cada perfil de cliente: 
        'Tipo de Cliente', 'Marca do Carro', 'Ano de Fabricação do Carro', 'Quilometragem do Carro no Momento da Venda', 'Tipo de Combustível','Tipo de Transmissão', 'Preço de Venda (R$)','Histórico de Aluguel (se aplicável)', 'Cancelamento da Venda'

Realizar com as seguintes colunas, para conseguir uma métrica de receita:
        'Data da Venda','Preço de Venda (R$)','Forma de Pagamento','Status do Pagamento', 'Desconto para Ex-cliente de Aluguel (%)', 'Cancelamento da Venda'

Realizar com as seguintes colunas, para conseguiur encontrar o perfil dos clientes mais valiosos:
        'Tipo de Cliente', 'Histórico de Aluguel (se aplicável)', 'Cancelamento da Venda', 'Motivo do Cancelamento (se aplicável)', 'Coerente'

Analise do porque ocorrem cancelamentos das compras:
        'Cancelamento da Venda', 'Motivo do Cancelamento (se aplicável)'

Apagar coluna coerente, True para todas as linhas

In [1]:
# Importação das bibliotecas para analise de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
db_vendas = pd.read_csv('historico_vendas.csv')

In [3]:
display(db_vendas.columns)
display(db_vendas.head())

Index(['ID do Cliente', 'Tipo de Cliente', 'Data da Venda',
       'ID do Carro Vendido', 'Marca do Carro', 'Ano de Fabricação do Carro',
       'Quilometragem do Carro no Momento da Venda', 'Tipo de Combustível',
       'Tipo de Transmissão', 'Preço de Venda (R$)', 'Forma de Pagamento',
       'Status do Pagamento', 'Histórico de Aluguel (se aplicável)',
       'ID do Contrato de Aluguel Anterior (se aplicável)',
       'Desconto para Ex-cliente de Aluguel (%)', 'Cancelamento da Venda',
       'Motivo do Cancelamento (se aplicável)', 'Coerente'],
      dtype='object')

,ID do Cliente,Tipo de Cliente,Data da Venda,ID do Carro Vendido,Marca do Carro,Ano de Fabricação do Carro,Quilometragem do Carro no Momento da Venda,Tipo de Combustível,Tipo de Transmissão,Preço de Venda (R$),Forma de Pagamento,Status do Pagamento,Histórico de Aluguel (se aplicável),ID do Contrato de Aluguel Anterior (se aplicável),Desconto para Ex-cliente de Aluguel (%),Cancelamento da Venda,Motivo do Cancelamento (se aplicável),Coerente
0,67593.0,b2c,13/05/2023,516713.0,voLKSWAGEn,2022.0,37656.0,HÍbridO,aUTOmÁTIcA,135375.00,TRanSfeRênCIA BanCáRIa,comPLETO,NÃo,NaN,0.0,NÃo,NaN,True
1,88684.0,B2b,03 de March de 2022,562471.0,cHEvRoLET,2013.0,214637.0,gaSoLIna,AuTomÁtICa,72948.61,FiNaNCIAmeNTo,EM AndamenTo,nãO,NaN,0.0,Não,NaN,True
2,60203.0,b2c,13/02/2023,367386.0,voLkSWaGEN,2016.0,134394.0,HÍBRiDO,AutoMáTIcA,89561.76,FInAncIAmEnto,em ANdAMEntO,nÃO,NaN,0.0,siM,FiNaNciAMEnTO NEgadO,True
3,40452.0,B2C,03 de August de 2023,126368.0,chEVrolet,2013.0,194853.0,híBrIDo,aUTOMÁticA,76788.01,TransFErência bANcÁRIa,cOmpLEto,NÃo,NaN,0.0,Não,NaN,True
4,83528.0,b2c,21/04/2022,988253.0,TOYoTA,2018.0,116024.0,eLÉtRiCo,AUtoMáTica,99237.41,finANcIameNto,Em AndAMeNtO,NÃO,NaN,0.0,NÃO,NaN,True


In [20]:

db_vendas['Coerente'].value_counts()

(23027, 18)

True    23007
Name: Coerente, dtype: int64

In [12]:
db_vendas['ID do Contrato de Aluguel Anterior (se aplicável)'].value_counts()

43670.0    4
26307.0    4
51782.0    3
61672.0    3
54858.0    3
          ..
81610.0    1
80229.0    1
54076.0    1
80481.0    1
3511.0     1
Name: ID do Contrato de Aluguel Anterior (se aplicável), Length: 5367, dtype: int64